<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/cu_cat_final_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# !pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11==22.10.0 cudf-cu11==22.10.0 cugraph-cu11==22.10.0 pylibraft_cu11==22.10.0 raft_dask_cu11==22.10.0 dask_cudf_cu11==22.10.0 pylibcugraph_cu11==22.10.0 pylibraft_cu11==22.10.0
# !pip install -U git+https://github.com/graphistry/pygraphistry.git@cudf_cat
# !pip install -U git+https://github.com/graphistry/cu-cat.git@cu_cat_regpt

In [ ]:
# !pip install dirty_cat

In [2]:
import cuml,cudf
cuml.__version__

'22.10.00a+119.gb30ef85c1'

In [5]:
!echo "0.01.0" > VERSION.txt
# !cp VERSION.txt /root/.local/lib/python3.9/site-packages/cu_cat/
!cp VERSION.txt /usr/local/lib/python3.9/dist-packages/cu_cat/

In [3]:
# from importlib import reload
# reload(cu_cat)
import cu_cat
print(cu_cat.__file__)

/usr/local/lib/python3.9/dist-packages/cu_cat/__init__.py


In [4]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 200)


In [7]:
import graphistry
graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='***') ## key id, secret key
graphistry.__version__


'0.28.7+328.gc2d2fcb'

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### redteam

In [8]:
df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
df['feats2'] = df.src_computer + ' ' + df.dst_computer
ndf = df.drop_duplicates(subset=['feats'])
tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
tdf['node'] = range(len(tdf))
tdf.RED.sum()

749.0

In [ ]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()

g = graphistry.nodes(cudf.from_pandas(tdf), 'node')

g5b = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
            #  remove_node_column=False,
            # engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cudf_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()

g = graphistry.nodes(tdf, 'node')

g5a = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
            #  remove_node_column=False,
            # engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()

g = graphistry.nodes((tdf), 'node')

g5c = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='dirty_cat',
            memoize=False,
            #  remove_node_column=False,
            # engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_dirty_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Using GPU: dirty_cat


### CTU-13 malware dataset

In [22]:
edf = pd.read_csv('https://gist.githubusercontent.com/silkspace/33bde3e69ae24fee1298a66d1e00b467/raw/dc66bd6f1687270be7098f94b3929d6a055b4438/malware_bots.csv', index_col=0)
T = edf.Label.apply(lambda x: True if 'Botnet' in x else False)
bot = edf[T]
nbot = edf[~T]
print(f'Botnet abundance: {100*len(bot)/len(edf):0.2f}%')# so botnet traffic makes up a tiny fraction of total

# let's balance the dataset in a 10-1 ratio, for speed and demonstrative purposes
negs = nbot.sample(10*len(bot))
edf = pd.concat([bot, negs])  # top part of arrays are bot traffic, then all non-bot traffic
edf = edf.drop_duplicates()

# some useful indicators for later that predict Botnet as Bool and Int
Y = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)  # np.array(T)

# Later we will use and exploit any meaning shared between the labels in a latent distribution

# add it to the dataframe
edf['bot'] = Y

# name some columns for edges and features
src = 'SrcAddr'
dst = 'DstAddr'
good_cols_with_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotBytes', 'SrcBytes', src, dst]

good_cols_without_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

## some encoding parameters
n_topics = 20
n_topics_target = 7



Botnet abundance: 0.69%


In [ ]:
# lets umap the data
g = graphistry.edges(cudf.from_pandas(edf), src, dst)

pr = cProfile.Profile()
pr.enable()
g6b = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            # use_scaler='quantile',
            # use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='cu_cat',
            # engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cudf-cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


In [ ]:
# lets umap the data
g = graphistry.edges(edf, src, dst)

pr = cProfile.Profile()
pr.enable()
g6a = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='cu_cat',
            # engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


In [ ]:
### lets umap the data
g = graphistry.edges(edf, src, dst)
pr = cProfile.Profile()
pr.enable()
g6c = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='dirty_cat',
            # engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


### 20newsgroups

In [34]:
from sklearn.datasets import fetch_20newsgroups
# newsgroups = fetch_20newsgroups()#categories=categories)
n_samples = 500
# n_features = 1000
# n_components = 10
# n_top_words = 20
# batch_size = 128
# init = "nndsvda"

news, _ = fetch_20newsgroups(
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"),
    return_X_y=True,
)

news = news[:n_samples]
news=pd.DataFrame(news)

In [32]:
import logging
# logging.basicConfig(level=logging)
logging.basicConfig(level=None)

In [35]:
g = graphistry.nodes(cudf.from_pandas(news))
pr = cProfile.Profile()
pr.enable()


g8 = g.featurize(feature_engine='cu_cat',memoize=False)
g8b = g8.umap()#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cudf-cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Featurizing nodes with feature_engine=cu_cat
Using GPU: cu_cat
         8512735 function calls (8496584 primitive calls) in 41.524 seconds

   Ordered by: internal time
   List reduced from 1951 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    6.291    2.097    6.291    2.097 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
        6    5.732    0.955    7.280    1.213 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1126(_multiplicative_update_h_smallfast)
      208    5.468    0.026   19.062    0.092 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1042(_multiplicative_update_h)
    84103    2.952    0.000    3.564    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
    13374    2.141    0.000    3.116    0.000 /usr/local/lib/python3.9/dist-packages/cupyx/scipy/sparse/_index.py:44(_get_csr_submatrix_major_axis)
    40606    1.971    0.00

In [37]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()

g7 = g.featurize(feature_engine='cu_cat',memoize=False)
g7a = g7.umap()#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Featurizing nodes with feature_engine=cu_cat
Using GPU: cu_cat


Using GPU: cu_cat


         17151891 function calls (17118991 primitive calls) in 91.635 seconds

   Ordered by: internal time
   List reduced from 2537 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       12   12.542    1.045   16.656    1.388 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1126(_multiplicative_update_h_smallfast)
      416   10.534    0.025   37.019    0.089 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1042(_multiplicative_update_h)
        6   10.250    1.708   10.250    1.708 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
        1    7.328    7.328    8.452    8.452 /usr/local/lib/python3.9/dist-packages/cuml/internals/api_decorators.py:397(inner_with_setters)
   168234    6.830    0.000    8.148    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
    26748    4.232    0.000    6.154    0.000 /usr/local/lib/python3.9/dist-packages/cupyx/sc

In [1]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()

g9 = g.featurize(feature_engine='dirty_cat',memoize=False)
# g9c = g9.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

NameError: ignored

In [ ]:
!cp *.txt /content/drive/MyDrive/graphistry

### winlogs

In [27]:
!wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet
winlogs=pd.read_parquet('part.88.parquet')
n=10000
winlogs=winlogs.sample(n,replace=False).iloc[:,:8]

g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()

g10 = g.featurize(feature_engine='cu_cat',memoize=False)
# g10a = g10.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize+umap_cu_cat_'+str(n)+'.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

g = graphistry.nodes(cudf.from_pandas(winlogs))
pr = cProfile.Profile()
pr.enable()

g11 = g.featurize(feature_engine='cu_cat',memoize=False)
# g11b = g11.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize+umap_cudf-cu_cat_'+str(n)+'.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()


g12 = g.featurize(feature_engine='dirty_cat',memoize=False)
# g12c = g12.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize+umap_dirtycat_'+str(n)+'.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

File ‘part.88.parquet’ already there; not retrieving.



Featurizing nodes with feature_engine=cu_cat
Using GPU: cu_cat
         5039507 function calls (4962708 primitive calls) in 34.820 seconds

   Ordered by: internal time
   List reduced from 2046 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       48   20.313    0.423   23.328    0.486 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1126(_multiplicative_update_h_smallfast)
       18    3.050    0.169    3.050    0.169 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
    18573    2.160    0.000    2.304    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
       42    1.843    0.044    2.237    0.053 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:181(_get_H)
       30    0.667    0.022    0.778    0.026 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:994(_multiplicative_update_w_smallfast)
     1895    0.428    0.000    0.459    0.000

Featurizing nodes with feature_engine=cu_cat
Using GPU: cu_cat
         5032028 function calls (4955295 primitive calls) in 35.162 seconds

   Ordered by: internal time
   List reduced from 1965 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       48   20.209    0.421   23.576    0.491 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1126(_multiplicative_update_h_smallfast)
       18    3.478    0.193    3.478    0.193 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
    18357    2.391    0.000    2.559    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
       42    1.798    0.043    2.190    0.052 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:181(_get_H)
       30    0.657    0.022    0.784    0.026 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:994(_multiplicative_update_w_smallfast)
    31174    0.417    0.000    0.595    0.000

In [ ]:
!cp *.txt /content/drive/MyDrive/graphistry

### ask HN

In [15]:
# # get the data top 3000 posts on Hacker News
askHN = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

# df = askHN.sample(1000,replace=True) # set smaller if you want to test a minibatch 
# df = df[['text','title']]
# df['title'] = df['title'].sample(frac=1).values

In [ ]:
g = graphistry.nodes(cudf.from_pandas(askHN))
pr = cProfile.Profile()
pr.enable()

g1 = g.featurize(feature_engine='cu_cat',memoize=False)
# g1a = g1.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cudf_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
# g1a = g1.umap(engine='cuml')

In [ ]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()

g2 = g.featurize(feature_engine='cu_cat',memoize=False)
# g2a = g2.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
# g2a = g2.umap(engine='cuml')

In [ ]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()


g3 = g.featurize(feature_engine='dirty_cat',memoize=False)
# g3a = g3.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_dirtycat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
# g1a.plot()

In [ ]:
# g1b.plot()